In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dill as pickle
sns.set_style('whitegrid')

import torch
from torch.autograd import Variable
from torch.utils import data
from torch import nn
from torch import optim
from torch.utils import data

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import RidgeCV

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

import sys
sys.path.append('..')#
from util import add_angles, angle_between, angled_vector, sub_angles
from mdn_model.mixture_loss import *
from mdn_model.models import *
from data import get_data

cuda:0


In [2]:
X_train, y_train, X_test, y_test = get_data(dir='../../data/processed/')

In [3]:
# Load mdn networks
covariance_type = 'diagonal'
n_hidden = 64
n_features = 192

models = []

checkpoint = torch.load('../../models/rnn_mdn.pt')
encoder = RNNEncoder(n_features=n_features, n_hidden=n_hidden)
decoder = MDNDecoder(n_hidden=n_hidden, n_components=5, covariance_type=covariance_type, covariance_reg=1e-6)
model_rnn_mdn = ReceptiveFieldNN(encoder=encoder, decoder=decoder).to(device)
model_rnn_mdn.load_state_dict(checkpoint['model'])
models += [{'model_name': 'rnn_mdn', 'model': model_rnn_mdn}]

checkpoint = torch.load('../../models/mlp_mdn.pt')
encoder = MLPEncoder(n_features=n_features, n_hidden=n_hidden)
decoder = MDNDecoder(n_hidden=n_hidden, n_components=5, covariance_type=covariance_type, covariance_reg=1e-6)
model_mlp_mdn = ReceptiveFieldNN(encoder=encoder, decoder=decoder).to(device)
model_mlp_mdn.load_state_dict(checkpoint['model'])
models += [{'model_name': 'mlp_mdn', 'model': model_mlp_mdn}]

In [4]:
def predict_mdn(model, data):
    pi, _, mu = [out.detach().cpu().numpy() for out in model(data.to(device))]
    return (np.stack((pi, pi), axis=-1) * mu).sum(axis=1)
    
def evaluate_mdn(model_name, model):
    model.eval()
    criterion = MixtureLoss().to(device)
    nll_train = criterion(*[o.double() for o in model(X_train.to(device))], y_train.to(device).double())
    nll_test = criterion(*model(X_test.to(device)), y_test.to(device))
    
    y_train_hat = predict_mdn(model, X_train)
    y_test_hat = predict_mdn(model, X_test)

    mse_train = mean_squared_error(y_true=y_train.numpy(),
                                  y_pred=y_train_hat)
    
    mse_test = mean_squared_error(y_true=y_test.numpy(),
                                  y_pred=y_test_hat)
       
    print(f"Model:\t {model_name}",
          f"NLL-Train:\t{nll_train}",
          f"NLL-Test:\t{nll_test}",
          f"MSE-Train:\t{mse_train}",
          f"MSE-Test:\t{mse_test}",
          "\n",
          sep='\n')

for model in models:
    evaluate_mdn(**model)

Model:	 rnn_mdn
NLL-Train:	1.4309556564481103
NLL-Test:	1.6875687837600708
MSE-Train:	0.43623560667037964
MSE-Test:	0.37120217084884644


Model:	 mlp_mdn
NLL-Train:	1.6296065007633673
NLL-Test:	1.597421407699585
MSE-Train:	0.447751522064209
MSE-Test:	0.3727840185165405




In [5]:
def compute_nll_mean(gt, predictions, train_variance):
    """Computes the negative log likelihood from mean predictions.
    Useful to estimate the nll for linear models"""
    # Model output as mixture of one Gaussian with diagonal cov matrix.
    criterion = MixtureLoss(covariance_type='diagonal')
    # Mean is given by predictions.
    mu_l = predictions.reshape(-1,1,2)
    # Pi is one (only one component!)
    pi_l = np.ones((predictions.shape[0],1))
    
    # Variance is same for each example (by homoscedasticity)
    sigma_l = np.tile((train_variance)[:,None], predictions.shape[0]).T.reshape(-1,1,2)
    return criterion(pi=torch.from_numpy(pi_l).double(),
                     sigma=torch.from_numpy(sigma_l).double(),
                     mu=torch.from_numpy(mu_l).double(),
                     y=torch.from_numpy(gt).double())

from train_linear import convert_data_concatenate, convert_data_no_memory
X_train_conc = convert_data_concatenate(X_train.numpy())
X_test_conc = convert_data_concatenate(X_test.numpy())

X_train_no_memory = convert_data_no_memory(X_train.numpy())
X_test_no_memory = convert_data_no_memory(X_test.numpy())

def evaluate_mse(model_name, model):
    if isinstance(model, nn.Module):
        model.eval()
        
    nll_train = None
    nll_test = None
    
    if 'conc' in model_name:
        X_train_linear = X_train_conc
        X_test_linear = X_test_conc
    else:
        X_train_linear = X_train_no_memory
        X_test_linear = X_test_no_memory
    
    if 'mse' in model_name:
        y_train_hat = model(X_train.to(device)).detach().cpu().numpy()
        y_test_hat = model(X_test.to(device)).detach().cpu().numpy()
    else:
        y_train_hat = model.predict(X_train_linear)
        y_test_hat = model.predict(X_test_linear)

    train_variance = np.var(y_train.numpy() - y_train_hat, axis=0)
    
    nll_train = compute_nll_mean(gt=y_train.numpy(),
                                predictions=y_train_hat,
                                train_variance=train_variance)
    nll_test = compute_nll_mean(gt=y_test.numpy(),
                            predictions=y_test_hat,
                            train_variance=train_variance)
    
    mse_train = mean_squared_error(y_true=y_train.numpy(),
                                  y_pred=y_train_hat)
    
    mse_test = mean_squared_error(y_true=y_test.numpy(),
                                  y_pred=y_test_hat)
    alpha = model.alpha_ if isinstance(model, RidgeCV) else ""
    if isinstance(model, RidgeCV):
        comment = model.alpha_
    elif isinstance(model, StaticSpatialLinearEncoder):
        comment = nn.functional.softmax(model.dt_weights, dim=0).cpu().detach().numpy()
    else:
        comment = ""
        
    print(f"Model:\t{model_name}",
          f"Comment:\t{comment}",
          f"NLL-Train:\t{nll_train}",
          f"NLL-Test:\t{nll_test}",
          f"MSE-Train:\t{mse_train}",
          f"MSE-Test:\t{mse_test}",
          "\n",
          sep='\n')
    
models = []

with open('../../models/linear_no_memory.model', 'rb') as file:
    linear_no_memory = pickle.load(file)
with open('../../models/linear_concatenate.model', 'rb') as file:
    linear_concatenate = pickle.load(file)   
    
models += [{'model_name': 'ridge_no_memory', 'model': linear_no_memory}]
models += [{'model_name': 'ridge_concatenate', 'model': linear_concatenate}]

checkpoint = torch.load('../../models/static_mse.pt')
model_static_mse = StaticSpatialLinearEncoder(n_features=n_features,
                                             n_dts=X_train.shape[0]).to(device)
model_static_mse.load_state_dict(checkpoint['model'])
models += [{'model_name': 'static_mse', 'model': model_static_mse}]

checkpoint = torch.load('../../models/mlp_mse.pt')
encoder = MLPEncoder(n_features=n_features, n_hidden=n_hidden)
decoder = NormalDecoder(n_hidden=n_hidden)
model_mlp_mse = ReceptiveFieldNN(encoder=encoder, decoder=decoder).to(device)
model_mlp_mse.load_state_dict(checkpoint['model'])
models += [{'model_name': 'mlp_mse', 'model': model_mlp_mse}]

checkpoint = torch.load('../../models/rnn_mse.pt')
encoder = RNNEncoder(n_features=n_features, n_hidden=n_hidden)
decoder = NormalDecoder(n_hidden=n_hidden)
model_rnn_mse = ReceptiveFieldNN(encoder=encoder, decoder=decoder).to(device)
model_rnn_mse.load_state_dict(checkpoint['model'])
models += [{'model_name': 'rnn_mse', 'model': model_rnn_mse}]

for model in models:
    evaluate_mse(**model)

Model:	ridge_no_memory
Comment:	19.916480638308585
NLL-Train:	2.041607232524099
NLL-Test:	1.8683679000952478
MSE-Train:	0.45139711445044906
MSE-Test:	0.37579735507351114


Model:	ridge_concatenate
Comment:	286.2153445389273
NLL-Train:	1.9986462498116737
NLL-Test:	1.8555734535565738
MSE-Train:	0.4323258718932058
MSE-Test:	0.37277552990758805


Model:	static_mse
Comment:	[0.04963709 0.05144349 0.05624986 0.06347179 0.07293913 0.07582849
 0.1051606  0.52526957]
NLL-Train:	2.0417163697996616
NLL-Test:	1.8646876465140054
MSE-Train:	0.4513101279735565
MSE-Test:	0.37361612915992737


Model:	mlp_mse
Comment:	
NLL-Train:	2.0420709678157287
NLL-Test:	1.8664833758808064
MSE-Train:	0.4516063332557678
MSE-Test:	0.37489619851112366


Model:	rnn_mse
Comment:	
NLL-Train:	1.9978073354843395
NLL-Test:	1.8648829612902957
MSE-Train:	0.4319530129432678
MSE-Test:	0.3767538070678711




In [6]:
y_pred_train_mean = y_train.mean(dim=0).repeat(y_train.shape[0],1).numpy()
y_pred_test_mean = y_train.mean(dim=0).repeat(y_test.shape[0],1).numpy()
print(mean_squared_error(y_train.numpy(), y_pred_train_mean),
mean_squared_error(y_test.numpy(), y_pred_test_mean))

train_variance_mean = np.var(y_train.numpy() - y_pred_train_mean, axis=0)
print(train_variance_mean)

nll_mean_train = compute_nll_mean(gt=y_train.numpy(),
                 predictions=y_pred_train_mean,
                 train_variance=train_variance_mean)

nll_mean_test = compute_nll_mean(gt=y_test.numpy(),
                 predictions=y_pred_test_mean,
                 train_variance=train_variance_mean)
print(nll_mean_train, nll_mean_test)

0.55788696 0.42263234
[0.5656871  0.55008703]
tensor(2.2542, dtype=torch.float64) tensor(2.0100, dtype=torch.float64)


In [7]:
np.cov(y_train.numpy() - y_pred_train_mean, rowvar=0)

array([[ 5.65718094e-01, -4.63753541e-05],
       [-4.63753541e-05,  5.50117386e-01]])